In [1]:
#coding: utf-8
from pymongo import MongoClient
from py2neo import Graph
import re
from pprint import pprint
client = MongoClient('localhost', 27017)
db = client.istex
coll_cambridge = db.cambridge
coll_springer = db.springer

In [2]:
import pandas as pd
import unicodedata

rnsr = pd.read_csv('rnsr.csv', sep=';', encoding='utf-8')
def strip_accents(c):
    s2 = unicodedata.normalize('NFD', c).encode('ascii', 'ignore')  
    return s2
columns = rnsr.columns
rnsr.columns = [strip_accents(c.replace(' ', '_').lower()) for c in columns]

def get_id_from_umr(rnsr, umr):
    f = rnsr[rnsr.iloc[:,2].fillna('UNKNOWN').str.contains(umr)].sort_values('annee')
    if f.groupby('numero_national_de_structure').nunique().shape[0] >= 1:
        return f.sort_values('annee').iloc[0,1]
    else:
        return None

def get_uai_tutelles(rnsr, id_rnsr):
    for uais in rnsr[rnsr['numero_national_de_structure'] == id_rnsr]['uai_des_tutelles'].tolist():
        for uai in str(uais).split(';'):
            yield uai

get_id_from_umr(rnsr, '7639')

u'199719338M'

In [3]:
%%time

import tqdm
# Getting authors from French affiliations with UMR
authors = []
publis = {}
aff = re.compile(".*UMR*", re.IGNORECASE)
def get_authors(coll):
    tot = coll.find({'author.affiliations': aff},{'author':1, 'id': 1, 'title': 1}).count()
    pbar = tqdm.tqdm(total=tot)
    for i in coll.find({'author.affiliations': aff},{'author':1, 'id': 1, 'title': 1}):
        pbar.update()
        tmp = 0
        for j in i['author']:
            if j.get('affiliations') and 'UMR' in j['affiliations'][0]:
                tmp = 1
                authors.append({'name': j['name'], 'umr': j['affiliations'][0], 'publis':i['id']})
        if tmp == 1:
            publis[i['id']] = {'id': i['id'], 'titre': i['title']}
get_authors(coll_cambridge)
get_authors(coll_springer)

 92%|█████████▏| 32920/35629 [00:08<00:23, 116.37it/s]

CPU times: user 1.42 s, sys: 148 ms, total: 1.57 s
Wall time: 17.2 s


In [4]:
import tqdm
reg_umr = re.compile("(UMR|CNRS)(| )(\d+)", re.IGNORECASE)
reg_digits = re.compile("\d{4}|\d{3}")
for a in tqdm.tqdm(authors):
    if reg_umr.search(a['umr']):
        a['umr'] = reg_umr.search(a['umr']).group(0).replace('-', ' ')
        if 'UMR ' in a['umr']:
            a['n_umr'] = reg_umr.search(a['umr']).group(3)
            a['id_rnsr'] = get_id_from_umr(rnsr, a['n_umr'])
        elif 'CNRS ' in a['umr']:
            a['umr'] = a['umr'].replace('CNRS', 'UMR')
            n = a['umr'].split(' ')[1]
            a['n_umr'] = n
            a['id_rnsr'] = get_id_from_umr(rnsr, a['umr'])
        elif 'UMR' in a['umr']:
            a['n_umr'] = a['umr'].split('UMR')[1]
            a['id_rnsr'] = get_id_from_umr(rnsr, n)
    elif reg_digits.search(a['umr']):
        a['n_umr'] = reg_digits.search(a['umr']).group(0)
        a['id_rnsr'] = get_id_from_umr(rnsr, a['n_umr'])



100%|██████████| 35629/35629 [00:12<00:00, 2920.37it/s]
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/sami/.virtualenvs/jupyter/local/lib/python2.7/site-packages/tqdm/_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "/home/sami/.virtualenvs/jupyter/lib/python2.7/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration


  4%|▍         | 286/6401 [00:25<09:02, 11.26it/s]


  9%|▉         | 588/6401 [00:52<08:37, 11.24it/s]


 14%|█▍        | 891/6401 [01:19<08:10, 11.24it/s]


 19%|█▊        | 1196/6401 [01:45<07:40, 11.31it/s]


 23%|██▎       | 1492/6401 [02:12<07:14, 11.29it/s]


...

 84%|████████▍ | 5367/6401 [07:54<01:31, 11.31it/s]


 88%|████████▊ | 5661/6401 [08:20<01:05, 11.30it/s]


 93%|█████████▎| 5959/6401 [08:47<00:39, 11.29it/s]


 98%|█████████▊| 6248/6401 [09:14<00:13, 11.26it/s]


100%|██████████| 6401/6401 [09:28<00:00, 11.26it/s]

In [5]:
from collections import Counter
total_none = Counter([x.get('id_rnsr') for x in authors])[None]
print('Pourcentage de RNSR match avec des chercheurs : %.2f%% sur %s chercheurs' %((100. - float(total_none)*100/len(authors)), len(authors)))

Pourcentage de RNSR match avec des chercheurs : 94.36% sur 6401 chercheurs


In [6]:
df_authors = pd.DataFrame(authors)
df_authors.head()

,id_rnsr,n_umr,name,publis,umr
0,199511953U,5509,M. C. JACOB,1A4FFDF078357DBE360A2BD90D8B291D2A23ED97,UMR 5509
1,200012191F,6118,ROMAIN VULLO,B8C893300DE6B8CFE14C9984749A0EB0049C2FA8,UMR 6118
2,199719338M,7639,F. Canova,156E50B6B4514DC38FA2FDE6299F1B6FEC72CF21,UMR 7639
3,199719338M,7639,A. Flacco,156E50B6B4514DC38FA2FDE6299F1B6FEC72CF21,UMR 7639
4,199719338M,7639,L. Canova,156E50B6B4514DC38FA2FDE6299F1B6FEC72CF21,UMR 7639


In [7]:
df_bce = pd.read_csv('uai.csv', delimiter=';', encoding='utf-8')
columns = df_bce.columns
df_bce.columns = [strip_accents(c.replace(' ', '_').lower()) for c in columns]
df_bce.head()

/home/sami/.virtualenvs/jupyter/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,siren,siret,libelle,sigle,siret_siege,date_de_creation,code_de_categorie,categorie,code_ape,libelle_ape,...,departement,code_de_la_region,region,code_du_pays,pays,geolocalisation,code_de_badge,badge,code_de_la_region_(avant_2016),region_(avant_2016)
0,193500972,NaN,Institut national des sciences appliquées de R...,Insa Rennes,1.935010e+13,1960-06-26,ENS_SUP,Enseignement supérieur,8542Z,Enseignement supérieur,...,Ille-et-Vilaine,53.0,Bretagne,NaN,NaN,"48.1216, -1.63297",anr;brevets;IDFI;LABX;wikipedia,Participant d'un projet financé par l'ANR;Dépo...,53.0,Bretagne
1,193113842,NaN,Université de Toulouse III - Paul Sabatier,NaN,1.931138e+13,1983-01-01,ENS_SUP,Enseignement supérieur,8542Z,Enseignement supérieur,...,Haute-Garonne,76.0,Languedoc-Roussillon - Midi-Pyrénées,NaN,NaN,"43.5608, 1.46936",anr;pcrdt;brevets;EQPX;EQPX;IDFI;LABX;PHUC;wik...,Participant d'un projet financé par l'ANR;Part...,73.0,Midi-Pyrénées
2,195903117,NaN,École nationale supérieure de chimie de Lille,ENSC Lille,1.959031e+13,NaN,ENS_SUP,Enseignement supérieur,8542Z,Enseignement supérieur,...,Nord,32.0,Nord-Pas-de-Calais - Picardie,NaN,NaN,"50.6083, 3.14808",anr;pcrdt;brevets;IEED;wikipedia,Participant d'un projet financé par l'ANR;Part...,31.0,Nord-Pas-de-Calais
3,196244016,NaN,Université d'Artois,NaN,1.962440e+13,1991-11-07,ENS_SUP,Enseignement supérieur,8542Z,Enseignement supérieur,...,Pas-de-Calais,32.0,Nord-Pas-de-Calais - Picardie,NaN,NaN,"50.2849, 2.79104",anr;pcrdt;brevets;IEED;LABX;wikipedia,Participant d'un projet financé par l'ANR;Part...,31.0,Nord-Pas-de-Calais
4,194200937,NaN,École nationale d'ingénieurs de Saint-Étienne,Enise,1.942009e+13,1983-01-01,ENS_SUP,Enseignement supérieur,8542Z,Enseignement supérieur,...,Loire,84.0,Auvergne - Rhône-Alpes,NaN,NaN,"45.4293, 4.42257",anr;pcrdt;brevets;EQPX;wikipedia,Participant d'un projet financé par l'ANR;Part...,82.0,Rhône-Alpes


In [9]:
%%time

from py2neo import Graph
from py2neo import Node
graph = Graph(user='neo4j', password='admin')

authors_dict = {}
rnsr_dict = {}
uai_dict = {}
publis_dict = {}
for etbl in df_bce[['code_uai', 'libelle', 'siren']].to_dict('records'):
    uai_dict[etbl['code_uai']] = Node('Etablissement',
                                          code_uai = etbl['code_uai'],
                                          siren = etbl['siren'],
                                          libelle = etbl['libelle'],
                                      )


for aff in rnsr[['numero_national_de_structure', 'annee', 'tutelles']].to_dict('records'):
    rnsr_dict[aff['numero_national_de_structure']] = Node('Affiliation',
                                                         id_rnsr = aff['numero_national_de_structure'],
                                                         tutelle = aff['tutelles'],
                                                         annee = aff['annee']
                                                         )
    
for aut in df_authors.to_dict('records'):
    authors_dict[aut['name']] = Node('Auteur',
                                    name = aut['name'],
                                    )
for i, pub in publis.items():
    publis_dict[i] = Node('Publication',
                         id = pub['id'],
                         titre = pub['titre'])

CPU times: user 10.2 s, sys: 552 ms, total: 10.8 s
Wall time: 10.7 s


In [10]:
%%time

for n in authors_dict.values():
    graph.create(n)
for n in uai_dict.values():
    graph.create(n)
for n in rnsr_dict.values():
    graph.create(n) 
for n in publis_dict.values():
    graph.create(n)

CPU times: user 20.1 s, sys: 1.58 s, total: 21.7 s
Wall time: 1min 19s


In [11]:
%%time

from py2neo import Relationship
links_authors = []
for aut in df_authors.dropna().to_dict('records'):
    a = authors_dict[aut['name']]
    rnsr_node = rnsr_dict[aut['id_rnsr']]
    graph.create(Relationship(a, 'affiliated', rnsr_node))
    for uai_code in get_uai_tutelles(rnsr, aut['id_rnsr']):
        if uai_code != 'nan':
            if uai_dict.get(uai_code):
                graph.create(Relationship(rnsr_node, 'tutelle', uai_dict[uai_code]))
for a in authors:
    p = publis[a['publis']]
    graph.create(Relationship(publis_dict[a['publis']],'auteur',authors_dict[a['name']]))

CPU times: user 4min 50s, sys: 15 s, total: 5min 5s
Wall time: 10min 49s


In [21]:
from fuzzywuzzy import fuzz

for i, a in enumerate(authors):
    for j in authors[i+1:]:
        score = fuzz.token_sort_ratio(a['name'], j['name'])
        if score > 85 and score != 100:
            print(u"Mots {} et {}, score: {}".format(a['name'], j['name'], score))

Mots F. Canova et L. Canova, score: 88
Mots Jean-Charles Martin et Jean-Charles Marty, score: 92
Mots Jean Vacelet et J. Vacelet, score: 86
Mots Jean Vacelet et J. Vacelet, score: 86
Mots Thierry Perez et Thierry Pérez, score: 96
Mots P. DURAND et S. Durand, score: 88
Mots Paul Guilloteau et P. Guilloteau, score: 89
Mots STEPHANE ZALESKI et STÉPHANE ZALESKI, score: 97
Mots Daniel Tomé et Daniel Thomas, score: 87
Mots PHILIPPE CARRIÈRE et Philippe Darriet, score: 88
Mots S. Morand et S. Morandat, score: 89
Mots MARIE-PIERRE DABARD et Marie-Pierre Béal, score: 86
Mots ANTHONY RANDRIAMAMPIANINA et A. Randriamampianina, score: 86
Mots J. F. GUEGAN et J.-F. GUÉGAN, score: 95
Mots Anna Trzeciakiewicz et A. Trzeciakiewicz, score: 91
Mots RÉEGIS MARCHIANO et Régis Marchiano, score: 97
Mots JOSEPH CASANOVA et Joseph Casanovas, score: 97
Mots A. Senger et B. Senger, score: 88
Mots MICHAEL LE BARS et Michaël Le Bars, score: 97
Mots P. COLLET et P. Coullet, score: 94
Mots P. COLLET et P. Coullet, 

Mots P. Bustamante et Paco Bustamante, score: 89
Mots P. Bustamante et Paco Bustamante, score: 89
Mots Pierre F. Monsan et Pierre Monsan, score: 93
Mots C. Leroux et C. Roux, score: 86
Mots Philippe Lenca et Lemanceau Philippe, score: 88
Mots Jérémie François et François Jérôme, score: 96
Mots Thierry Denceux et Thierry Denœux, score: 93
Mots J. Chevalier et Y. Chevalier, score: 91
Mots J. Chevalier et Y. Chevalier, score: 91
Mots J. Chevalier et Y. Chevalier, score: 91
Mots M. Ayrault et C. Ayrault, score: 89
Mots Josette Olivier-Fourcade et J. Olivier-Fourcade, score: 86
Mots Josette Olivier-Fourcade et J. Olivier-Fourcade, score: 86
Mots Josette Olivier-Fourcade et J. Olivier-Fourcade, score: 86
Mots Josette Olivier-Fourcade et J. Olivier-Fourcade, score: 86
Mots Nicolas D. Lindley et Nicholas D. Lindley, score: 97
Mots Jean Paul Barthe et Jean-Paul A. Barthès, score: 88
Mots C. Gaimon et C. Guimon, score: 88
Mots Huyên Pham et Huyen Pham, score: 95
Mots Sylvie Rousset et Sylvie Dou

Mots C. Gérardin et C. Gérard, score: 88
Mots Zoï Kapoula et Z. Kapoula, score: 95
Mots Lemanceau Philippe et Philippe Lenca, score: 88
Mots Paul Manneville et P. Manneville, score: 89
Mots M. Gibert et J. Gibert, score: 88
Mots C. Lemaire et E. Lemaire, score: 89
Mots Zo Kapoula et Z. Kapoula, score: 95
Mots S. Ahzi et Saïd Ahzi, score: 86
Mots O. Rougier et M. Rougier, score: 89
Mots H. Bourgeois et F. Bourgeois, score: 91
Mots C. Rey et C. Reylé, score: 91
Mots Olivier Teytuad et Olivier Teytaud, score: 93
Mots Reinoud J. Bootsma et Reinoud Bootsma, score: 94
Mots J. Desmaison et M. Desmaison, score: 91
Mots Jean Pailhous et J. Pailhous, score: 87
